In [1]:
# imports and froms
import pandas as pd
import pymysql
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()

In [2]:
# load CSVs into dataframes
dog_csv = 'Resources/dog_intelligence.csv'
dog_df = pd.read_csv(dog_csv, encoding = 'utf8')
dog_df.head()

,Breed,Classification,obey,reps_lower,reps_upper
0,Border Collie,Brightest Dogs,95%,1,4
1,Poodle,Brightest Dogs,95%,1,4
2,German Shepherd,Brightest Dogs,95%,1,4
3,Golden Retriever,Brightest Dogs,95%,1,4
4,Doberman Pinscher,Brightest Dogs,95%,1,4


In [3]:
# load CSVs into dataframes
hetero60_csv = 'Resources/Table_5_Expected_Heterozygosity_60_breeds.csv'
hetero60_df = pd.read_csv(hetero60_csv, encoding = 'utf8')
hetero60_df.columns = ['Breed', 'Expected_Hetero']
hetero60_df.head()

,Breed,Expected_Hetero
0,Scottish Deerhound,2.0683
1,Field Spaniel,2.3165
2,Flat-coated Retriever,2.6474
3,Bernese Mountain Dog,2.8129
4,Standard Schnauzer,2.8129


In [4]:
# load CSVs into dataframes
hetero85_csv = 'Resources/Table_4_Heterozygosity_85_breeds.csv'
hetero85_df = pd.read_csv(hetero85_csv, encoding = 'utf8')
hetero85_df.head()

,Population,Heterozygosit
0,Bedlington Terrier,0.312842
1,Miniature Bull Terrier,0.321619
2,Boxer,0.343151
3,Clumber Spaniel,0.363595
4,Greater Swiss Mountain Dog,0.364943


In [5]:
# load CSVs into dataframes
akc_csv = 'Resources/AKC Breed Info.csv'
akc_df = pd.read_csv(akc_csv, encoding='latin1')
akc_df.head()

,Breed,height_low_inches,height_high_inches,weight_low_lbs,weight_high_lbs
0,Akita,26,28,80,120
1,Anatolian Sheepdog,27,29,100,150
2,Bernese Mountain Dog,23,27,85,110
3,Bloodhound,24,26,80,120
4,Borzoi,26,28,70,100


In [6]:
# declare connection variables for mypysql and sqlalchemy
host = '127.0.0.1' 
user = 'root'
password = ''
db = 'etl_db'
charset = 'utf8'
port = 3306
cursorclass = pymysql.cursors.DictCursor

In [7]:
# establish mypymsql connection
connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    charset=charset,
    cursorclass=cursorclass)
cursor_object = connection.cursor()

In [8]:
# create database
sql_create = (f'CREATE DATABASE IF NOT EXISTS {db}')  
cursor_object.execute(sql_create)

# show databases
sql_query = 'SHOW DATABASES'
cursor_object.execute(sql_query)
db_list = cursor_object.fetchall()

for database in db_list:
    print(database)

{'Database': 'etl_db'}
{'Database': 'information_schema'}
{'Database': 'mysql'}
{'Database': 'performance_schema'}
{'Database': 'sakila'}
{'Database': 'sys'}
{'Database': 'world'}


In [9]:
# reestablish pymysql connection with database specified
connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    db=db, 
    charset=charset,
    cursorclass=cursorclass)
cursor_object = connection.cursor()

In [10]:
# create tables
query_list = [
    'CREATE TABLE IF NOT EXISTS akc(Breed VARCHAR(32), height_low_inches INT, height_high_inches INT, weight_low_lbs INT, weight_high_lbs INT)',
    'CREATE TABLE IF NOT EXISTS dogs(Breed VARCHAR(32), Classification VARCHAR(32), obey FLOAT, reps_lower INT, reps_upper INT)',
    'CREATE TABLE IF NOT EXISTS hetero60(Breed VARCHAR(32), Heterozygosity FLOAT)',
    'CREATE TABLE IF NOT EXISTS hetero85(Population VARCHAR(32), Heterozygosit FLOAT)'
]

for query in query_list:
    cursor_object.execute(query)

# show tables
sql_query = 'SHOW TABLES'   
cursor_object.execute(sql_query)
rows = cursor_object.fetchall()

for row in rows:
    print(row)

{'Tables_in_etl_db': 'akc'}
{'Tables_in_etl_db': 'dogs'}
{'Tables_in_etl_db': 'hetero60'}
{'Tables_in_etl_db': 'hetero85'}


In [11]:
# establish sqlalchemy connection
con = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8', echo=False)

# load data into tables
akc_df.to_sql(name='akc', con=con, if_exists='replace', index=False)
dog_df.to_sql(name='dogs', con=con, if_exists='replace', index=False)
hetero85_df.to_sql(name='hetero85', con=con, if_exists='replace', index=False)
hetero60_df.to_sql(name='hetero60', con=con, if_exists='replace', index=False)

In [12]:
# reestablish pymysql connection after loading data
connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    db=db, 
    charset=charset,
    cursorclass=cursorclass)
cursor_object = connection.cursor()

# show data in tables
query_list = [
    'SELECT * FROM akc LIMIT 10',
    'SELECT * FROM dogs LIMIT 10',
    'SELECT * FROM hetero60 LIMIT 10',
    'SELECT * FROM hetero85 LIMIT 10'
]

for query in query_list:
    cursor_object.execute(query)
    rows = cursor_object.fetchall()

    for row in rows:
        print(row)
        
    print(end='\n')

{'Breed': 'Akita', 'height_low_inches': '26', 'height_high_inches': '28', 'weight_low_lbs': '80', 'weight_high_lbs': '120'}
{'Breed': 'Anatolian Sheepdog', 'height_low_inches': '27', 'height_high_inches': '29', 'weight_low_lbs': '100', 'weight_high_lbs': '150'}
{'Breed': 'Bernese Mountain Dog', 'height_low_inches': '23', 'height_high_inches': '27', 'weight_low_lbs': '85', 'weight_high_lbs': '110'}
{'Breed': 'Bloodhound', 'height_low_inches': '24', 'height_high_inches': '26', 'weight_low_lbs': '80', 'weight_high_lbs': '120'}
{'Breed': 'Borzoi', 'height_low_inches': '26', 'height_high_inches': '28', 'weight_low_lbs': '70', 'weight_high_lbs': '100'}
{'Breed': 'Bullmastiff', 'height_low_inches': '25', 'height_high_inches': '27', 'weight_low_lbs': '100', 'weight_high_lbs': '130'}
{'Breed': 'Great Dane', 'height_low_inches': '32', 'height_high_inches': '32', 'weight_low_lbs': '120', 'weight_high_lbs': '160'}
{'Breed': 'Great Pyrenees', 'height_low_inches': '27', 'height_high_inches': '32', '

In [13]:
# join all four tables on dog breed
query = ['SELECT akc.Breed, akc.height_low_inches, akc.height_high_inches, akc.weight_low_lbs, akc.weight_high_lbs, dogs.Classification, dogs.obey, dogs.reps_lower, dogs.reps_upper, hetero85.Heterozygosit, hetero60.Expected_Hetero FROM akc LEFT JOIN dogs ON akc.Breed=dogs.Breed LEFT JOIN hetero60 ON akc.Breed=hetero60.Breed LEFT JOIN hetero85 ON akc.Breed=hetero85.Population'
]

cursor_object.execute(query[0])
rows = cursor_object.fetchall()

for row in rows:
    print(row)

{'Breed': 'Bedlington Terrier', 'height_low_inches': '15', 'height_high_inches': '16', 'weight_low_lbs': '18', 'weight_high_lbs': '23', 'Classification': 'Average Working/Obedience Intelligence', 'obey': '50%', 'reps_lower': 26, 'reps_upper': 40, 'Heterozygosit': 0.312842, 'Expected_Hetero': 3.9712}
{'Breed': 'Boxer', 'height_low_inches': '21', 'height_high_inches': '25', 'weight_low_lbs': '65', 'weight_high_lbs': '70', 'Classification': 'Average Working/Obedience Intelligence', 'obey': '50%', 'reps_lower': 26, 'reps_upper': 40, 'Heterozygosit': 0.343151, 'Expected_Hetero': 3.0611}
{'Breed': 'Clumber Spaniel', 'height_low_inches': '19', 'height_high_inches': '20', 'weight_low_lbs': '35', 'weight_high_lbs': '65', 'Classification': 'Above Average Working Dogs', 'obey': '70%', 'reps_lower': 16, 'reps_upper': 25, 'Heterozygosit': 0.363595, 'Expected_Hetero': None}
{'Breed': 'Doberman Pinscher', 'height_low_inches': '26', 'height_high_inches': '28', 'weight_low_lbs': '60', 'weight_high_lbs'

In [14]:
# terminate pymysql connection
connection.close()